In [28]:
import pandas as pd
import numpy as np
import redditcleaner 

ticker_df = pd.read_csv("tickers_and_names.csv")
ticker_df = ticker_df[["ticker", "name"]]
ticker_df["name"] = ticker_df["name"].str.replace("&#39;", "'")

df = pd.read_csv('wsb_truncated.csv', parse_dates=["created_utc"], engine='python', encoding='utf-8', error_bad_lines=False)
# Get relative frequencies/hotness per trading DAY, and then plot it relative to whatever

Skipping line 1127325: unexpected end of data


In [29]:
df.head(5)

,id,created_utc,selftext,title
0,msci36,2021-04-16 17:05:07-04:00,NaN,🔥🔥🔥5 DAY AMC BORROW FEE %🔥🔥🔥
1,mschp6,2021-04-16 17:04:34-04:00,[removed],Dance of Darkness: The SEC and Darkpools.
2,mschoq,2021-04-16 17:04:33-04:00,NaN,Had to do it on opening day
3,mschjq,2021-04-16 17:04:21-04:00,NaN,Greetings from the Reeperbahn
4,msch0a,2021-04-16 17:03:41-04:00,[removed],EVBox (TPGY) - Are they a real competitor of C...


In [31]:
df = df.replace(np.nan, '', regex=True)
df = df.replace({r'[removed]' : ''}, regex=True)
# df3 = df2.replace({'\n' : ' '}, regex=True)
# df4 = df3.replace({'&amp;' : '&'}, regex=True)
# df5 = df4.replace({'#x200B;' : ' '}, regex=True)
# df5.head(20)

df['selftext'] = df['selftext'].map(redditcleaner.clean)
df['title'] = df['title'].map(redditcleaner.clean)
df.head(10)

,id,created_utc,selftext,title
0,sci36,2021-04-16 17:05:07-04:00,,🔥🔥🔥5 DAY AMC BORROW FEE %🔥🔥🔥
1,schp6,2021-04-16 17:04:34-04:00,[],Danc f Daknss: Th SEC an Dakpls.
2,schq,2021-04-16 17:04:33-04:00,,Ha t it n pning ay
3,schjq,2021-04-16 17:04:21-04:00,,Gtings f th Rpbahn
4,sch0a,2021-04-16 17:03:41-04:00,[],EVBx (TPGY) - A thy a al cptit f Chagpint (CHP...
5,scgua,2021-04-16 17:03:28-04:00,,$CLOV Unusual Optins Actiity UOA
6,scgnx,2021-04-16 17:03:14-04:00,[],Taking XRP high
7,scfp,2021-04-16 17:01:52-04:00,"Hi t anyn wh taks th ti t a this, I a tying t ...",Hlping y a n last ti
8,scf4,2021-04-16 17:01:51-04:00,"TLDR: GME t th n. I a in GME, AMC, PLTR, an a ...","WWk, WMn??"
9,scfx,2021-04-16 17:01:31-04:00,,Fllw aps I’ sy I can’t aff than 0.28 sha But I...


In [32]:
ticker_df.head()

,ticker,name
0,A,"Agilent Technologies, Inc."
1,AA,Alcoa Corporation
2,AABA,Altaba Inc.
3,AAC,"AAC Holdings, Inc."
4,AAL,"American Airlines Group, Inc."


In [33]:
stock_tickers = list(ticker_df["ticker"]) 
# Might be missing some stopwords here, but this should be decently robust 
# A combination of [link1] and [link2]
stop_tickers = ["YOLO", "PUMP", "RH", "EOD", "IPO", "ATH", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
blacklist_words = [
      "YOLO", "TOS", "CEO", "CFO", "CTO", "DD", "BTFD", "WSB", "OK", "RH",
      "KYS", "FD", "TYS", "US", "USA", "IT", "ATH", "RIP", "BMW", "GDP",
      "OTM", "ATM", "ITM", "IMO", "LOL", "DOJ", "BE", "PR", "PC", "ICE",
      "TYS", "ISIS", "PRAY", "PT", "FBI", "SEC", "GOD", "NOT", "POS", "COD",
      "AYYMD", "FOMO", "TL;DR", "EDIT", "STILL", "LGMA", "WTF", "RAW", "PM",
      "LMAO", "LMFAO", "ROFL", "EZ", "RED", "BEZOS", "TICK", "IS", "DOW"
      "AM", "PM", "LPT", "GOAT", "FL", "CA", "IL", "PDFUA", "MACD", "HQ",
      "OP", "DJIA", "PS", "AH", "TL", "DR", "JAN", "FEB", "JUL", "AUG",
      "SEP", "SEPT", "OCT", "NOV", "DEC", "FDA", "IV", "ER", "IPO", "RISE"
      "IPA", "URL", "MILF", "BUT", "SSN", "FIFA", "USD", "CPU", "AT",
      "GG", "ELON", "EV"
]
stop_tickers = set(stop_tickers).union(set(blacklist_words))
stock_tickers = [ticker for ticker in stock_tickers if ticker not in stop_tickers]

# In the future, also parse for stock names. Right now compute is a bottleneck
# stock_names = list(ticker_df["name"])

In [34]:
print(df.iloc[0]["title"])

🔥🔥🔥5 DAY AMC BORROW FEE %🔥🔥🔥


In [35]:
# Merge selftext and title into one column 
df['text'] = df.selftext.astype(str).str.cat(df.title.astype(str), sep='. ')
df = df.drop(columns=["selftext", "title"])
df.head()

,id,created_utc,text
0,sci36,2021-04-16 17:05:07-04:00,. 🔥🔥🔥5 DAY AMC BORROW FEE %🔥🔥🔥
1,schp6,2021-04-16 17:04:34-04:00,[]. Danc f Daknss: Th SEC an Dakpls.
2,schq,2021-04-16 17:04:33-04:00,. Ha t it n pning ay
3,schjq,2021-04-16 17:04:21-04:00,. Gtings f th Rpbahn
4,sch0a,2021-04-16 17:03:41-04:00,[]. EVBx (TPGY) - A thy a al cptit f Chagpint ...


In [36]:
print(df.iloc[7]["text"])

Hi t anyn wh taks th ti t a this, I a tying t hlp y fath wh cntly pass away 12-4-21 t ha a gat sn ff by aising funs f a funal, y fath in't ha lif insuanc an ys ppl will slat this pst but I want t ty what I cul thinking that it can nly tak n psn t hlp sn. I wul b than happy t pi any infatin that wul shw this is a al pst an a al situatin. https://www.justgiing.c/cwfuning/kin-lilly. Hlping y a n last ti


In [37]:
date_df = df.copy()
date_df["created_utc"] = pd.to_datetime(df['created_utc'], utc=True)
date_df["created_utc"] = date_df["created_utc"].dt.date.astype("datetime64")
date_df.tail()

,id,created_utc,text
1127318,g1tts,2020-04-15,What th hll is ging n. My wif will n t a sugay...
1127319,g1tu,2020-04-15,. I want t put y whl stiulus chck n a call put...
1127320,g1tuf,2020-04-15,. Gay pnis club
1127321,g1tuic,2020-04-15,Dug ha a link t this akt watch aticl tay...Pub...
1127322,g1tuf,2020-04-15,. AMZN laps 💎 🤚 upat: 270k pfit


In [38]:
# Count POSTS mentioning a stock
# In the future, could probably weight this by the activity a post gets 

from collections import defaultdict
from pprint import pprint

def inner_dict():
    return {}

# mydict = lambda: defaultdict(inner_dict)

def nested_dict():
    return lambda: defaultdict(inner_dict)

dates = date_df.created_utc.unique()
# historical_list = [_ for _ in range(len(dates[0:2]))]
historical_list = mydict()

for i, date in enumerate(dates[0:2]):
#     daily_mentions = mydict()
    filtered_df = date_df[date_df["created_utc"].isin(pd.date_range(date, date))][:10]
    for j, ticker in enumerate(stock_tickers):
        # name = stock_names[j]
        # Returns a Series of eaxct-match counts in the length of the number of submissions on the given day
        count = filtered_df.text.str.count(f'(?<!\\S){ticker}(?!\\S)')
        historical_list[ticker][date] = sum(count)
#         print(daily_mentions)
    
#     historical_list[i] = daily_mentions
#     daily_df = pd.DataFrame(daily_mentions)
#     pprint(daily_mentions)
#     display(daily_df)
    
historical_df = pd.DataFrame(historical_list)
display(historical_df)
        
# def strip_nested_dict(some_dict):
#     new_dict = {}
#     for sub_dict_key, sub_dict in some_dict.items():
#         new_dict[sub_dict_key] = {}
#         for key, value in sub_dict.items():
#             if value != 0:
#                 new_dict[sub_dict_key][key] = value
#     return new_dict

# temp = []
# for stock in stock_mentions:
#     for date in stock_mentions[stock]:
#         if stock_mentions[stock][date] > 0:
#             temp.append((stock, stock_mentions[stock][date]))
            
# pprint(sorted(temp, key=lambda x: x[1])[::-1])

,AA,AABA,AAC,AAL,AAMC,AAME,AAN,AAOI,AAON,AAP,...,ZTEST,ZTO,ZTR,ZTS,ZUMZ,ZUO,ZVO,ZYME,ZYNE,ZYXI
2021-04-16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-04-15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
